In [5]:
import numpy as np
import pandas as pd
import string

from itertools import product
from sklearn import preprocessing

from subprocess import check_output
from scipy.stats import pearsonr

from os import listdir
from os.path import isfile, join

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Weighted Average Stacking
* 여러가지 방법으로 학습한 결과를 마지막에 결과가 서로 상관관계가 적거나 혹은 결과가 좋은 Model 여러개를 사용하여 결과를 합침<br>
* 아래 사이트를 참고하였고 간단하게는 결과를 합친 뒤 평균을 내는 것 같아 이 코드를 작성함
<br>https://mlwave.com/kaggle-ensembling-guide/
* 이 코드에서 각 결과에 가중치를 주는 방법은 결과 파일을 폴더 내에 몇 번 Copy하냐에 따라서 가중치의 계수 결정됨<br>
**Example**<br>
A,B,C,D 결과가 있을 경우 A에 가중치 3을 주고 싶을 때 A를 Copy하여 A', A''를 만들어주면 (A+A'+A''+B+C+D)/6 으로 계산됨

**사용 방법**<br>
>1. StackNet 혹은 다른 방법으로 나온 결과 파일을 한 폴더에 모아 놓는다.
>2. stacking_path에 폴더 Path를 입력
>3. 아래 순서로 실행하고 마지막에 결과파일의 이름을 변경하여 제출한다.

In [21]:
stacking_path = 'ensemble/submit/E9/'

In [3]:
df = pd.DataFrame(columns=['low','medium','high','listing_id'])

In [22]:
df = pd.DataFrame()

In [23]:
file_list = []
file_path_list=[]
try:
    for file in listdir(stacking_path):
        
        if file.find('.csv')==-1:
            continue
            
        print(file)    
        file_list.append(file)
        file_path = join(stacking_path,file)
        file_path_list.append(file_path)
        temp = pd.read_csv(file_path)
        if df.empty == True:
            print("empty")
            df = df.append(temp)
        else:
            print("in")
            df = df.merge(temp,on='listing_id')
except:
    print("exception")

E10_folder_ensemble_20170426_1.csv
empty
E1_folder_ensemble_20170423_5.csv
in
E2_folder_ensemble_20170424_2.csv
in
E3_folder_ensemble_20170425_5.csv
in
E4_folder_ensemble_20170425_2.csv
in
E5_folder_ensemble_20170425_3.csv
in
E7_folder_ensemble_20170426_1.csv
in
E8_folder_ensemble_20170426_3 - 복사본 (2).csv
in
E8_folder_ensemble_20170426_3 - 복사본.csv
in
E8_folder_ensemble_20170426_3.csv
in


In [24]:
if 'low' in df.columns:
    df['low_result'] = df['low_x'].mean(axis=1) + df['low_y'].mean(axis=1) + df['low']
    df['medium_result'] = df['medium_x'].mean(axis=1) + df['medium_y'].mean(axis=1) + df['medium']
    df['high_result'] = df['high_x'].mean(axis=1) + df['high_y'].mean(axis=1) + df['high']

    df['low_result'] = df['low_result']/3
    df['medium_result'] = df['medium_result']/3
    df['high_result'] = df['high_result']/3
    
    del df['low_x'],df['low_y'],df['medium_x'],df['medium_y'],df['high_x'],df['high_y'],df['high'],df['medium'],df['low']
    
else:
    df['low_result'] = df['low_x'].mean(axis=1) + df['low_y'].mean(axis=1) 
    df['medium_result'] = df['medium_x'].mean(axis=1) + df['medium_y'].mean(axis=1) 
    df['high_result'] = df['high_x'].mean(axis=1) + df['high_y'].mean(axis=1)

    df['low_result'] = df['low_result']/2
    df['medium_result'] = df['medium_result']/2
    df['high_result'] = df['high_result']/2
    
    del df['low_x'],df['low_y'],df['medium_x'],df['medium_y'],df['high_x'],df['high_y']

In [25]:
df.rename(columns={'low_result':'low','medium_result':'medium','high_result':'high'},inplace=True)
df['listing_id'] = df['listing_id'].astype(int)

In [26]:
df.to_csv(join(stacking_path,'E9_folder_ensemble_20170426_1.csv'),index=False)

# Calculate Correlation

In [15]:
stacking_base_file = 'E8_folder_ensemble_20170426_3.csv'

In [16]:
stacking_path = 'ensemble/submit/E9/'

In [17]:
def calculate_correlation(base_df,target):
    source = base_df.copy()
    source = source.merge(target,on='listing_id')
    corr_df = source.corr()
    low_corr = corr_df.ix['low_x']['low_y']
    medium_corr = corr_df.ix['medium_x']['medium_y']
    high_corr = corr_df.ix['high_x']['high_y']
    
    del corr_df,source
    return low_corr,medium_corr,high_corr, (low_corr+medium_corr+high_corr)/3

In [18]:
def is_same_listing_id(source,target):
    return 1 if np.sum(source['listing_id'] - target['listing_id']) == 0 else 0

In [19]:
file_list = []
file_path_list=[]
base_df = pd.read_csv(join(stacking_path,stacking_base_file))
corr_dict = dict()
try:
    for file in listdir(stacking_path):
        
        if file.find('.csv')==-1:
            continue
            
        print(file)    
        file_list.append(file)
        file_path = join(stacking_path,file)
        file_path_list.append(file_path)
        #print(file_path)
        temp = pd.read_csv(file_path)

        if is_same_listing_id(base_df,temp) == 0:    
            continue

        l,m,h,c = calculate_correlation(base_df,temp)
        corr_dict[file]=c
        print(file,c)
        print("")
except:
    print("exception")

E10_folder_ensemble_20170426_1.csv
E10_folder_ensemble_20170426_1.csv 0.997999462643

E1_folder_ensemble_20170423_5.csv
E1_folder_ensemble_20170423_5.csv 0.994741764178

E2_folder_ensemble_20170424_2.csv
E2_folder_ensemble_20170424_2.csv 0.994714336959

E3_folder_ensemble_20170425_5.csv
E3_folder_ensemble_20170425_5.csv 0.994911320929

E4_folder_ensemble_20170425_2.csv
E4_folder_ensemble_20170425_2.csv 0.996427804291

E5_folder_ensemble_20170425_3.csv
E5_folder_ensemble_20170425_3.csv 0.996272126743

E7_folder_ensemble_20170426_1.csv
E7_folder_ensemble_20170426_1.csv 0.999937168391

E8_folder_ensemble_20170426_3 - 복사본 (2).csv
E8_folder_ensemble_20170426_3 - 복사본 (2).csv 1.0

E8_folder_ensemble_20170426_3 - 복사본.csv
E8_folder_ensemble_20170426_3 - 복사본.csv 1.0

E8_folder_ensemble_20170426_3.csv
E8_folder_ensemble_20170426_3.csv 1.0



In [72]:
sorted(corr_dict,key=corr_dict.get)

['pred_ex64_nineth.csv',
 'pred_ex60_ensemble.csv',
 'top_corr_ensemble_base_before_top.csv',
 'top5_ensemble.csv',
 'top5_ensemble_base_before_top.csv',
 'top5_ensemble_20170418.csv',
 'just_voting1.csv',
 'top5_2_result_with_nineth.csv',
 'top5_2_result_without_nineth.csv',
 'top5_2_result - 복사본.csv',
 'top5_2_result.csv',
 'top5_2_result - 복사본 (2).csv']